In [1]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
# from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random
import tfutils as t


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
class DCGAN():
    
    def __init__(self):
        self.gf_dim = 64
        self.df_dim = 64
        self.height = 128
        self.width = 128
        self.channel = 3
        self.z_dim = 128
        self.lr = 1e-4
        self.beta1 = 0.5
        self.sample_num = 5
        tf.reset_default_graph()
        self.sess = tf.Session()
        self.batch_size = 10
        # Placeholders
        self.x = tf.placeholder(tf.float32, shape=[None, self.height, self.width, self.channel], name='x-images')
        self.z = tf.placeholder(tf.float32, shape=[None, self.z_dim], name='z-noise')

        self.IMG_SHAPE = (self.width, self.height, 3)
    
    def extract_data(self):
        train = []
        test = []
        
        if os.path.exists(np_train_path):
            
            print("\n..........loading dataset from numpy files..........\n")
            
            with gzip.GzipFile(np_train_path, "r") as f:
                train = np.load(f)
        else:
            
            print("\n..........loading dataset from disk..........\n")
            
            for file in os.listdir("./airplane"):
                file_path = os.path.join("./airplane", file)
                if str(file) == "train":
                    for label in sorted(os.listdir(file_path)):
                        label_path = os.path.join(file_path, label)
                        for img in os.listdir(label_path):
                            img_path = os.path.join(label_path, img)
                            if str(img[0:-4]) == "1":
                                train1 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                            elif str(img[0:-4]) == "6":
                                train2 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                            elif str(img[0:-4]) == "10":
                                train3 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        merged = [train1, train2, train3]
                        random.shuffle(merged)
                        image1 = cv.merge(tuple(merged))
#                         image1 = cv.merge((train1, train2, train3))
                        train.append(image1)
        #                 cv.imshow("image", image)
        #                 cv.waitKey(0)
        #                 cv.destroyAllWindows()
        #                 cv.imwrite("./2.jpg", image1)
        #                 break

    #             elif str(file) == "test":
    #                 for label in sorted(os.listdir(file_path)):
    #                     label_path = os.path.join(file_path, label)
    #                     for img in os.listdir(label_path):
    #                         img_path = os.path.join(label_path, img)
    #                         if str(img[0:-4]) == "1":
    #                             test1 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    #                         elif str(img[0:-4]) == "6":
    #                             test2 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    #                         elif str(img[0:-4]) == "10":
    #                             test3 = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    #                     image2 = cv.merge((test1, test2, test3))
    #                     test.append(image2)
            train = np.array(train, dtype="float") / 255.0
    #         test = np.array(test, dtype="float") / 255.0
            train = train.reshape(train.shape[0], 128, 128, 3)
    #         test = test.reshape(test.shape[0], 128, 128, 3)

            with gzip.GzipFile(np_train_path, "w") as f:
                np.save(f, train)
    #         with gzip.GzipFile(np_test_path, "w") as f:
    #             np.save(f, test)
        return train
    
    # gf_dim * value represents total number of conv filters in that layer, i.e, 64 * 8, 64 * 4, etc.
    def generator(self, z, reuse=None, is_train=True):
        with tf.variable_scope('generator', reuse=reuse):
            x = t.dense(z, self.gf_dim * 16 * 4 * 4, name='gen-fc-1')

            x = tf.reshape(x, [-1, 4, 4, self.gf_dim * 16])
            x = t.batch_norm(x, is_train=is_train, name='gen-bn-1')
            x = tf.nn.relu(x)
            
            x = t.deconv2d(x, self.gf_dim * 8, 5, 2, name='gen-deconv2d-1')
            x = t.batch_norm(x, is_train=is_train, name='gen-bn-2')
            x = tf.nn.relu(x)
#             print(x)

            x = t.deconv2d(x, self.gf_dim * 4, 5, 2, name='gen-deconv2d-2')
            x = t.batch_norm(x, is_train=is_train, name='gen-bn-3')
            x = tf.nn.relu(x)
#             print(x)

            x = t.deconv2d(x,  self.gf_dim * 2, 5, 2, name='gen-deconv2d-3')
            x = t.batch_norm(x, is_train=is_train, name='gen-bn-4')
            x = tf.nn.relu(x)
#             print(x)

            x = t.deconv2d(x,  self.gf_dim * 1, 5, 2, name='gen-deconv2d-4')
            x = t.batch_norm(x, is_train=is_train, name='gen-bn-5')
            x = tf.nn.relu(x)
#             print(x)
            
            x = t.deconv2d(x, self.channel, 5, 2, name='gen-deconv2d-5')
            x = tf.nn.tanh(x)
#             print(x)

            return x
        
        
    def discriminator(self, x, reuse=None):
        
        vgg = tf.contrib.keras.applications.VGG16(input_shape=self.IMG_SHAPE, input_tensor=x, weights="imagenet", include_top=False)
#             output = tf.identity(vgg.block5_pool, name='my_output')
        vgg.trainable = False
        output = vgg.layers[-1].output
        
        with tf.variable_scope('discriminator', reuse=reuse):
            
#             x = t.conv2d(output, self.df_dim * 1, 5, 2, name='disc-conv2d-1')
#             x = tf.nn.leaky_relu(x)

#             x = t.conv2d(x, self.df_dim * 2, 5, 2, name='disc-conv2d-2')
#             x = t.batch_norm(x, name='disc-bn-1')
#             x = tf.nn.leaky_relu(x)

#             x = t.conv2d(x, self.df_dim * 4, 5, 2, name='disc-conv2d-3')
#             x = t.batch_norm(x, name='disc-bn-2')
#             x = tf.nn.leaky_relu(x)

#             x = t.conv2d(x, self.df_dim * 8, 5, 2, name='disc-conv2d-4')
#             x = t.batch_norm(x, name='disc-bn-3')
#             x = tf.nn.leaky_relu(x)

            x = tf.layers.flatten(output)

            x = t.dense(x, 256, name='disc-fc-1')
            x = tf.nn.leaky_relu(x)
            
            x = t.dense(x, 128, name='disc-fc-2')
            x = tf.nn.leaky_relu(x)
            
            logits = t.dense(x, 1, name='disc-fc-3')
#             print(logits)
            
            prob = tf.nn.sigmoid(logits) # only to check output manually, not given to optimizer
            
            return prob, logits
        
        
    def build_dcgan(self):
        
        # Generator
        self.g = self.generator(self.z)

#         # Discriminator
        _, d_real = self.discriminator(self.x)
        _, d_fake = self.discriminator(self.g, reuse=True)
        
        # Losses
        d_real_loss = t.sce_loss(d_real, tf.ones_like(d_real))
        d_fake_loss = t.sce_loss(d_fake, tf.zeros_like(d_fake))
        self.d_loss = d_real_loss + d_fake_loss
        self.g_loss = t.sce_loss(d_fake, tf.ones_like(d_fake))
        
        # Collect trainer values
        t_vars = tf.trainable_variables()
        d_params = [v for v in t_vars if v.name.startswith('d')]
        g_params = [v for v in t_vars if v.name.startswith('g')]

        # Optimizer
        self.d_op = tf.train.AdamOptimizer(learning_rate=1e-6,
                                           beta1=self.beta1).minimize(self.d_loss, var_list=d_params)
        self.g_op = tf.train.AdamOptimizer(learning_rate=1e-4,
                                           beta1=self.beta1).minimize(self.g_loss, var_list=g_params)
        
    def train_model(self):
        
        train = dcgan.extract_data()
        dcgan.build_dcgan()

        total_epochs = 350

        dataset = tf.data.Dataset.from_tensor_slices(train)
        dataset = dataset.shuffle(len(train)).repeat().batch(self.batch_size)
        iterator = dataset.make_one_shot_iterator()
        x_data = iterator.get_next()

        self.sess.run(tf.global_variables_initializer())
        
        print("------------Start of training---------------")

        for epoch in range(1, total_epochs):

            for j in range(int(len(train)/self.batch_size)):
                x_batch = self.sess.run(x_data)
                z_batch = np.random.uniform(-1., 1., [self.batch_size, self.z_dim]).astype(np.float32)
                _, dis_loss = self.sess.run([self.d_op, self.d_loss], feed_dict={self.x: x_batch, self.z: z_batch})
                _, gen_loss = self.sess.run([self.g_op, self.g_loss], feed_dict={self.x: x_batch, self.z: z_batch})

            print("Epoch %d, d_loss: %g, g_loss: %g" %(epoch, dis_loss, gen_loss))

            if epoch % 50 == 0:
                # Training G model with sample image and noise
                sample_z = np.random.uniform(-1., 1., [self.sample_num, self.z_dim])
                samples = self.sess.run(self.g, feed_dict={self.z: sample_z})
                # Generated image save
                for s in range(samples.shape[0]):
                    cv.imwrite("./airplane_gen/"+str(epoch)+"_"+str(s)+".jpg", np.array(samples[s] * 255.0))
#                 t.save_images(samples,
#                                size=[self.height, self.width],
#                                image_path="./airplane/"+,
#                                inv_type='127')
                
        self.sess.close()
            

In [4]:
dcgan = DCGAN()
np_train_path = './train.npy.gz'
# np_test_path = './test.npy.gz'
# dcgan.discriminator(dcgan.x)
dcgan.train_model()
    


..........loading dataset from numpy files..........

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Use keras.layers.flatten instead.
------------Start of training---------------
Epoch 1, d_loss: 1.39525, g_loss: 0.683991
Epoch 2, d_loss: 1.39484, g_loss: 0.684308
Epoch 3, d_loss: 1.39068, g_loss: 0.688213
Epoch 4, d_loss: 1.3865, g_loss: 0.692177
Epoch 5, d_loss: 1.38224, g_loss: 0.696085
Epoch 6, d_loss: 1.37989, g_loss: 0.697991
Epoch 7, d_loss: 1.37786, g_loss: 0.699438
Epoch 8, d_loss: 1.37618, g_loss: 0.700552
Epoch 9, d_loss: 1.37518, g_loss: 0.700765
Epoch 10, d_loss: 1.37423, g_loss: 0.700857
Epoch 11, d_loss: 1.37321, g_loss: 0.701029
Epoch 12, d_loss: 1.37316, g_loss: 0.700514
Epoch 13, d_loss: 1.37285, g_loss: 0.699881
Epo

KeyboardInterrupt: 